In [2]:
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

inputText = open_file('data\Switzerland.txt')
inputText = inputText.replace('\n', ' ').replace('\r', '')

In [3]:
import tiktoken

def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

tokenizer = tiktoken.get_encoding('cl100k_base')
tiktoken_len(inputText)

16674

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=150,
        chunk_overlap=20,
        length_function=tiktoken_len,
        separators=["\n\n", "\n", " ", ""]
    )

chunks = text_splitter.split_text(inputText)

In [5]:
from langchain.vectorstores import Chroma
docsearch = Chroma.from_texts(chunks)

Using embedded DuckDB without persistence: data will be transient
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction
C:\Users\jbury_000\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
query = "Who was Zwingli"
docsearch.similarity_search(query)

[Document(page_content='Swiss Reformation. Against what he viewed as the decadent Roman Catholic hierarchy, Zwingli favoured the return to the teachings of the Bible. While Luther strictly separated the spiritual and political realms, Zwingli emphasized that both the church and the state were subject to the law of Christ. In 1525 Zürich’s great council adopted his innovations: the Latin', metadata={}),
 Document(page_content='Renaissance scholar Desiderius Erasmus, whose Christian philosophy became the heart of humanism in Switzerland. One of Erasmus’s most eager pupils was Huldrych Zwingli, an influential theologian and a dynamic political leader whose new Protestant religious doctrines, paralleling to some extent those ofMartin Luther, fueled the Swiss Reformation. Against what he viewed as the', metadata={}),
 Document(page_content='foreigners regarded it as an entity. \x0cThe ancien régime  The Reformation        Huldrych Zwingli  Switzerland’s then biggest town, Basel, became a cu

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
OPENAI_API_KEY = open_file('openaiapikey.txt')
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

In [8]:
qa.run('Who was Zwingli?')

'Huldrych Zwingli was an influential theologian and a dynamic political leader whose new Protestant religious doctrines, paralleling to some extent those of Martin Luther, fueled the Swiss Reformation. Against what he viewed as the decadent Roman Catholic hierarchy, Zwingli favored the return to the teachings of the Bible.'

In [9]:
qa.run('What was Zwingli first name?')

"Huldrych was Zwingli's first name."